In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_hdf('test.h5','data')
train = pd.read_hdf('train.h5','data')

In [68]:

def maxtimeGroup(df):
    gr = df.groupby('user_id')
    predict = np.zeros(len(gr),dtype=[('user_id','<i4'),('product_id','<i4')])
    for i,(user_id, ur) in enumerate(gr):
        r,ri, rc = np.unique(ur[['product_id']][::-1],return_counts=True,return_index=True)
        selectur = ur[['event_time','product_id']][::-1].iloc[ri]
        selectur['count'] = rc
        sur = np.sort(selectur.to_records(index=False),order=['count','event_time'])
        predict[i] = (user_id, sur[-1]['product_id'])
    return predict

In [69]:
predict = maxtimeGroup(test)

In [53]:
def lasttimeGroup(df):
    gr = df.groupby('user_id')
    predict = np.zeros(len(gr),dtype=[('user_id','<i4'),('product_id','<i4')])
    for i,(user_id, ur) in enumerate(gr):
        predict[i] = (user_id, ur.iloc[-1]['product_id'])
    return predict

In [70]:
pd.DataFrame(predict).to_csv('item.csv',index=False)

In [54]:
lastpredict=lasttimeGroup(test)

In [55]:
pd.DataFrame(lastpredict).to_csv('item1.csv',index=False)

In [75]:
def maxtimeGradeGroup(df):
    gr = df.groupby('user_id')
    predict = np.zeros(len(gr),dtype=[('user_id','<i4'),('product_id','<i4')])
    for i,(user_id, ur) in enumerate(gr):
        r,ri, rc = np.unique(ur[['product_id']][::-1],return_counts=True,return_index=True)
        grade = ur.groupby('product_id')['event_type'].sum()
        selectur = ur[['event_time','product_id']][::-1].iloc[ri]
        selectur['count'] = rc
        selectur = selectur.merge(grade,on='product_id')
        sur = np.sort(selectur.to_records(index=False),order=['event_type','event_time','count'])
        
        predict[i] = (user_id, sur[-1]['product_id'])
    return predict

In [ ]:
test.loc[:,('event_type')] = test['event_type'].apply(lambda x: -10 if x==10 else x)

In [76]:
gradepredict=maxtimeGradeGroup(test)

In [77]:
pd.DataFrame(gradepredict).to_csv('item2.csv',index=False)

In [78]:
test.loc[:,('event_type')] = test['event_type'].apply(lambda x: 2 if x==1 else x)

In [79]:
gradepredict=maxtimeGradeGroup(test)
pd.DataFrame(gradepredict).to_csv('item3.csv',index=False)